# ABS Monthly International Trade 5368

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import matplotlib.pyplot as plt
import readabs as ra
from readabs import metacol as mc

# local imports
from abs_helper import get_abs_data
from plotting import seas_trend_plot

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("5302.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

## Plot

### Headline charts

In [3]:
def headline_charts() -> None:
    """Produce headline charts."""

    tab = "530204"  # seasonally adjusted and trend estimates
    meta_subset = meta[(meta[mc.table] == tab)]
    headline = meta_subset[mc.did].unique()

    for h in headline:
        # secure the data
        f = pd.DataFrame()
        for service_type in "Seasonally Adjusted", "Trend":
            search = {
                h: mc.did,
                service_type: mc.stype,
            }
            try:
                table, series_id, units = ra.find_abs_id(meta_subset, search)
            except ValueError:
                continue
            f[service_type] = abs_dict[table][series_id]
        if len(f) == 0:
            continue
        f_, units = ra.recalibrate(f, units, verbose=False)

        # plot
        title = (
            h.replace("Credits", "Credits (Exports)")
            .replace("Debits", "Debits (Imports)")
            .replace(" ;", "")
            .strip()
        )
        long_line = 50
        if "," in title and len(title) > long_line:
            title = title.replace(",", ",\n")
        lfooter = "Australia. Current Prices. "
        if f_["Trend"].isna().iloc[-1]:
            last = f_["Trend"].last_valid_index()
            lfooter += f"No trend data after {last!r}. "

        seas_trend_plot(
            pd.DataFrame(f_),  # mypy
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            show=SHOW,
        )


headline_charts()

## Finished

In [4]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Jul 16 2024 09:25:34

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

pandas    : 2.2.2
matplotlib: 3.9.1
readabs   : 0.0.10a4

Watermark: 2.4.3



In [5]:
print("Finished")

Finished
